In [1]:
import numpy as np
import tflearn
import tflearn.activations as activations
# Data loading and preprocessing
from tflearn.activations import relu
from tflearn.data_utils import shuffle, to_categorical
from tflearn.layers.conv import avg_pool_2d, conv_2d, max_pool_2d
from tflearn.layers.core import dropout, flatten, fully_connected, input_data
from tflearn.layers.merge_ops import merge
from tflearn.layers.normalization import local_response_normalization
from tflearn.layers.normalization import batch_normalization


#chris library imports
from matplotlib import pyplot as plt
import cv2
import tensorflow as tf
from sklearn.model_selection import train_test_split

from test_recursive_image_load_V2 import load_CKP_data
from test_recursive_image_load_V2 import load_formated_data
from showNumpyInfo import showInfo


# global Paths to define for each specific computer
#tf_checkpoints = where the checkpoints of tensorflow training algorithms are stored to be recovered if necessary
tf_checkpoints = "G:/Documenten/personal/school/MaNaMA_AI/thesis/implementation/dexpression/github_1/github/Thesis_Christiaan_Jean/Custom_Dexpression/tf_checkpoints"

# cascPath = the path to the cascade file for the facerecognition (relative paths didn't work on my windows edition)
cascPath = "G:/Documenten/personal/school/MaNaMA_AI/thesis/implementation/dexpression/github_1/github/Thesis_Christiaan_Jean/Custom_Dexpression/haarcascade.xml"

# Give a run ID here. Change it to flags (arguments) in version 2.
ID = '4_2'
RUNID = 'DeXpression_run_' + ID

# Give a dropout if required (change to True and define the dropout percentage).
dropout = False
dropout_keep_prob=0.5

# Load data from: https://drive.google.com/drive/folders/1YWT8DJivNOZzQRPCiHDPY0LL_dymdQIS?usp=sharing
X_data = np.load('CKP_X.npy')
Y_data = np.load('CKP_Y.npy')
X_subID = (np.load('CKP_subjectIds.npy')).astype('uint8')

X_reduced = X_data.astype('uint8')
Y_reduced = Y_data

# showInfo(X_reduced,"X_reduced")
# showInfo(Y_reduced,"Y_reduced")

# showInfo(X_subID,"X_subID")



hdf5 is not supported on this machine (please install/reinstall h5py for optimal experience)
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [2]:
print(X_subID)

# seperate the dataset in 11 parts that do not share users: 
#   9 parts train set
#   1 part validation set
#   1 part test set
# allows future possibility for 10-fold crossover

# X_parts = np.asarray([np.asarray([]), np.asarray([]), np.asarray([]), np.asarray([]), np.asarray([]), np.asarray([]), np.asarray([]), np.asarray([]), np.asarray([]),np.asarray([]), np.asarray([])])
X_parts = []
# print(type(X_parts))
# print(type(X_parts[0]))
# print(len(X_parts))
# print(X_parts)
# print(X_parts[0])

# Y_parts =  np.asarray([np.asarray([]), np.asarray([]), np.asarray([]), np.asarray([]), np.asarray([]), np.asarray([]), np.asarray([]), np.asarray([]), np.asarray([]),np.asarray([]), np.asarray([])])
# X_subID_parts =  np.asarray([np.asarray([]), np.asarray([]), np.asarray([]), np.asarray([]), np.asarray([]), np.asarray([]), np.asarray([]), np.asarray([]), np.asarray([]),np.asarray([]), np.asarray([])])
Y_parts = []
X_subID_parts = []  
                         
# showInfo(X_parts ,"X_parts")
# showInfo(Y_parts ,"Y_parts")    
    
X_subID_unique,X_subID_unique_ID,X_subID_count  = np.unique(X_subID,return_index= True,return_counts=True)
X_subID_cum = np.cumsum(X_subID_count)
# print("X_subID_unique")
# print(X_subID_unique)
# print("X_subID_unique_ID")
# print(X_subID_unique_ID)
# print("X_subID_count")
# print(X_subID_count)
# print("X_subID_cum")  
# print(X_subID_cum)

# cv2.imshow("example", X_reduced[0].reshape((224,224)))
# cv2.waitKey(0)




        

[  5  10  10  10  11  11  11  11  11  11  14  14  14  14  22  22  22  26
  26  26  26  28  29  32  32  32  32  32  34  34  34  35  35  35  37  37
  37  42  42  42  42  44  44  44  45  45  46  46  46  46  50  50  50  50
  51  51  52  52  52  53  53  54  54  54  55  55  55  55  55  56  56  56
  57  57  57  58  58  58  59  59  60  60  60  61  61  61  62  62  62  62
  63  63  64  64  64  65  65  65  65  66  66  66  66  67  67  67  67  68
  68  68  68  69  69  69  70  70  70  71  71  71  71  71  72  72  73  73
  74  74  74  74  75  75  75  75  76  76  76  77  77  78  78  78  79  79
  79  80  80  80  81  81  81  82  82  82  83  84  84  85  85  85  86  86
  87  87  87  87  88  88  89  89  89  90  90  90  91  91  92  92  92  93
  93  94  94  95  95  95  95  96  96  96  97  97  97  98  98  99  99  99
 100 100 100 101 102 102 102 105 106 106 106 107 107 108 108 108 109 109
 109 110 111 111 111 112 113 113 113 114 114 115 115 115 116 116 116 117
 117 117 119 119 119 122 124 124 124 124 125 125 12

In [3]:
partSize = (np.floor(len((X_subID))/12)).astype('uint8')
print("partsize " , partSize)

lastSubID = 0 #the ID of the subject of the last instance
partCount = 0 #The amount of instances in this part already
partID = 0    #keeps the number of the part that is being filled right now

X_array = np.asarray([])
Y_array = np.asarray([])
sub_array = np.asarray([])

X_parts = []
# print(type(X_parts))
# print(type(X_parts[0]))
# print(len(X_parts))
# print(X_parts)
# print(X_parts[0])

# Y_parts =  np.asarray([np.asarray([]), np.asarray([]), np.asarray([]), np.asarray([]), np.asarray([]), np.asarray([]), np.asarray([]), np.asarray([]), np.asarray([]),np.asarray([]), np.asarray([])])
# X_subID_parts =  np.asarray([np.asarray([]), np.asarray([]), np.asarray([]), np.asarray([]), np.asarray([]), np.asarray([]), np.asarray([]), np.asarray([]), np.asarray([]),np.asarray([]), np.asarray([])])
Y_parts = []
X_subID_parts = []  

#iterate through al the ID's if a part is full and the subjectID is different from the previous, fill the next part 
for i in range(0,len(X_subID)):

#     print("this part IDs top border ", partSize ," and partCount = ", partCount)
    if((partCount >= partSize) & (lastSubID != X_subID[i])):
      
        
        #fill the parts to the partslists
        X_parts.append(X_array)
        Y_parts.append(Y_array)
        X_subID_parts.append(sub_array)
        
        #refresh the temporary arrays
        X_array = np.asarray([])
        Y_array = np.asarray([])
        sub_array = np.asarray([])
        #reset counter & set next part to be filled
        partCount = 0
        partID = partID + 1
        
        
    X_array = np.append(X_array ,X_reduced[i])
    Y_array = np.append(Y_array ,Y_reduced[i])
    sub_array = np.append(sub_array ,X_subID[i])  
    

    partCount = partCount +1
    lastSubID = X_subID[i]

print(len(X_parts))
print(len(Y_parts))
print(len(X_subID_parts))

partsize  27
11
11
11


In [4]:
# print(len(X_subID))
# print("partsize", partSize)

for o in range(0,len(X_parts)):
    print (o)
    showInfo(X_parts[o].reshape(-1,224,224,1))
    showInfo(Y_parts[o].reshape(-1,7))
    showInfo(X_subID_parts[o])

0
Name  UnKnown
type:  <class 'numpy.ndarray'>
Dtype:  float64
shape:  (28, 224, 224, 1)
Name  UnKnown
type:  <class 'numpy.ndarray'>
Dtype:  float64
shape:  (28, 7)
Name  UnKnown
type:  <class 'numpy.ndarray'>
Dtype:  float64
shape:  (28,)
1
Name  UnKnown
type:  <class 'numpy.ndarray'>
Dtype:  float64
shape:  (28, 224, 224, 1)
Name  UnKnown
type:  <class 'numpy.ndarray'>
Dtype:  float64
shape:  (28, 7)
Name  UnKnown
type:  <class 'numpy.ndarray'>
Dtype:  float64
shape:  (28,)
2
Name  UnKnown
type:  <class 'numpy.ndarray'>
Dtype:  float64
shape:  (27, 224, 224, 1)
Name  UnKnown
type:  <class 'numpy.ndarray'>
Dtype:  float64
shape:  (27, 7)
Name  UnKnown
type:  <class 'numpy.ndarray'>
Dtype:  float64
shape:  (27,)
3
Name  UnKnown
type:  <class 'numpy.ndarray'>
Dtype:  float64
shape:  (28, 224, 224, 1)
Name  UnKnown
type:  <class 'numpy.ndarray'>
Dtype:  float64
shape:  (28, 7)
Name  UnKnown
type:  <class 'numpy.ndarray'>
Dtype:  float64
shape:  (28,)
4
Name  UnKnown
type:  <class 'numpy

In [5]:
#select 9 from the 11 parts (part 11 is usually the smallest so it is bad as test or validation data)
select =  [0,1,2,3,6,7,8,9,10]
select_val = 4
select_test= 5

if((len(select)+1 + 1)!= len(X_parts)):
    print("Warning it is possible that not all parts of the dataset are used")

X = np.asarray([])
Y = np.asarray([])
subID = np.asarray([])
    
for i in select:
    X = np.append(X,X_parts[i])
    Y = np.append(Y,Y_parts[i])
    subID = np.append(subID,X_subID_parts[i])

X = (X.reshape(-1,224,224,1)).astype('uint8')
Y = (Y.reshape(-1,7)).astype('uint8')
subID = subID.astype('uint8')
    
X_val = X_parts[select_val].reshape(-1,224,224,1).astype('uint8')
Y_val = Y_parts[select_val].reshape(-1,7).astype('uint8')
subID_val = X_subID_parts[select_val].astype('uint8')

X_test = X_parts[select_test].reshape(-1,224,224,1).astype('uint8')
Y_test = Y_parts[select_test].reshape(-1,7).astype('uint8')
subID_test = X_subID_parts[select_test].astype('uint8')

showInfo(X,"X")
showInfo(Y,"Y")
showInfo(X_val,"X_vall")
showInfo(Y_val,"Y_vall")
showInfo(X_test,"X_test")
showInfo(Y_test,"Y_test")

# for i in range(0,len(X)):
#     str = "index " + repr(i) + " X  "+ repr(subID[i]) + " emotion " +  repr(Y[i])
#     print(str)
#     cv2.imshow(str, X[i].reshape((224,224)))
#     cv2.waitKey(0)
# cv2.destroyAllWindows()


Name  X
type:  <class 'numpy.ndarray'>
Dtype:  uint8
shape:  (249, 224, 224, 1)
Name  Y
type:  <class 'numpy.ndarray'>
Dtype:  uint8
shape:  (249, 7)
Name  X_vall
type:  <class 'numpy.ndarray'>
Dtype:  uint8
shape:  (28, 224, 224, 1)
Name  Y_vall
type:  <class 'numpy.ndarray'>
Dtype:  uint8
shape:  (28, 7)
Name  X_test
type:  <class 'numpy.ndarray'>
Dtype:  uint8
shape:  (27, 224, 224, 1)
Name  Y_test
type:  <class 'numpy.ndarray'>
Dtype:  uint8
shape:  (27, 7)


In [6]:
showInfo(X , "X")

# cv2.imshow("example", X[1].reshape((224,224)))
# cv2.waitKey(0)

print("size labels shape: " + repr(Y.shape))
print("type labels: " + repr(type(Y)))


Name  X
type:  <class 'numpy.ndarray'>
Dtype:  uint8
shape:  (249, 224, 224, 1)
size labels shape: (249, 7)
type labels: <class 'numpy.ndarray'>


In [7]:
# Define number of output classes.
num_classes = 7

# Define padding scheme.
padding = 'VALID'

# Model Architecture
network = input_data(shape=[None, 224, 224, 1])
conv_1 = relu(conv_2d(network, 64, 7, strides=2, bias=True, padding=padding, activation=None, name='Conv2d_1'))
maxpool_1 = batch_normalization(max_pool_2d(conv_1, 3, strides=2, padding=padding, name='MaxPool_1'))
#LRN_1 = local_response_normalization(maxpool_1, name='LRN_1')
# FeatEX-1
conv_2a = relu(conv_2d(maxpool_1, 96, 1, strides=1, padding=padding, name='Conv_2a_FX1'))
maxpool_2a = max_pool_2d(maxpool_1, 3, strides=1, padding=padding, name='MaxPool_2a_FX1')
conv_2b = relu(conv_2d(conv_2a, 208, 3, strides=1, padding=padding, name='Conv_2b_FX1'))
conv_2c = relu(conv_2d(maxpool_2a, 64, 1, strides=1, padding=padding, name='Conv_2c_FX1'))
FX1_out = merge([conv_2b, conv_2c], mode='concat', axis=3, name='FX1_out')
# FeatEX-2
conv_3a = relu(conv_2d(FX1_out, 96, 1, strides=1, padding=padding, name='Conv_3a_FX2'))
maxpool_3a = max_pool_2d(FX1_out, 3, strides=1, padding=padding, name='MaxPool_3a_FX2')
conv_3b = relu(conv_2d(conv_3a, 208, 3, strides=1, padding=padding, name='Conv_3b_FX2'))
conv_3c = relu(conv_2d(maxpool_3a, 64, 1, strides=1, padding=padding, name='Conv_3c_FX2'))
FX2_out = merge([conv_3b, conv_3c], mode='concat', axis=3, name='FX2_out')
net = flatten(FX2_out)
if dropout:
    net = dropout(net, dropout_keep_prob)
loss = fully_connected(net, num_classes,activation='softmax')

# Compile the model and define the hyperparameters
network = tflearn.regression(loss, optimizer='Adam',
                     loss='categorical_crossentropy',
                     learning_rate=0.0001)

#create a custom tensorflow session to manage the used resources
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config = config)


# Final definition of model checkpoints and other configurations
#model = tflearn.DNN(network, checkpoint_path='/home/cc/DeXpression/DeXpression_checkpoints',
model = tflearn.DNN(network, checkpoint_path=tf_checkpoints,
                    max_checkpoints=1, tensorboard_verbose=2, tensorboard_dir="./tflearn_logs/")

Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [8]:
tf.RunOptions(report_tensor_allocations_upon_oom = True)

report_tensor_allocations_upon_oom: true

In [9]:
# Fit the model, train for 20 epochs. (Change all parameters to flags (arguments) on version 2.)
#model.fit(X, Y, n_epoch=20, validation_set=0.1, shuffle=True, show_metric=True, batch_size=50, snapshot_step=2000,snapshot_epoch=True, run_id=RUNID)
model.fit(X, Y, n_epoch=20, validation_set=(X_test,Y_test), shuffle=True, show_metric=True, batch_size=50, snapshot_step=2000,snapshot_epoch=True, run_id=RUNID)

# Save the model
model.save(tf_checkpoints + '/' + RUNID + '.model')
print("finished training and saving")

# Load the model if required, later.
#model.load('./DeXpression_checkpoints/' + RUNID + '.model')

Training Step: 99  | total loss: 1.02537 | time: 18.071s
| Adam | epoch: 020 | loss: 1.02537 - acc: 0.8902 -- iter: 200/249
Training Step: 100  | total loss: 1.74276 | time: 24.281s
| Adam | epoch: 020 | loss: 1.74276 - acc: 0.8231 | val_loss: 0.79043 - val_acc: 0.7407 -- iter: 249/249
--
finished training and saving
